In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

#Get page links
def get_urls(ui,n):  #ui:links，n：Page number
    urllst = []
    for i in range(1,n+1):
        urllst.append(ui+str(i))
    return urllst

#Get 10 basic information per page
def get_onepage_data(u):  #u:The url
    r = requests.get(u)
    soup = BeautifulSoup(r.text, 'lxml')
    infor = soup.find('ul',class_ = "list_item clrfix").find_all('li')
    data_jd = []
    data_link = []
    for i in infor:
        dic = {}
        dic_link = {}
        dic['lat'] = i['data-lat']
        dic['lng'] = i['data-lng']
        dic['attraction_names'] = i.find('span', class_="cn_tit").text
        dic['strategy_sum'] = i.find('div', class_="strategy_sum").text
        dic['comment_sum'] = i.find('div', class_="comment_sum").text
        dic['ranking'] = i.find('span', class_="ranking_sum").text
        dic['travelers_sum'] = i.find('span', class_="sum").text.split('%')[0]
        dic['star'] = i.find('span', class_="cur_star")['style'].split(':')[1].split('%')[0]
        dic['description'] = i.find('div', class_="desbox").text
        dic['url'] = i.find('a')['href']
        data_jd.append(dic)        
    return data_jd
    
#Get n pages of web information
def get_all_data(urls): #urls Url list
    data_list= []
    
    for i in urls:       
        data_list.extend(get_onepage_data(i))        
    #Import the DataFrame of pandas 
    df = pd.DataFrame(data_list)     
    df.index = df['attraction_names']
    del df['attraction_names']               
    return df

#Data characters are converted to Numbers for computational processing
def data_collation(df):      
    df['lng'] = df['lng'].astype(np.float)
    df['lat'] = df['lat'].astype(np.float)
    df['comment_sum'] = df['comment_sum'].astype(np.int)
    df['strategy_sum'] = df['strategy_sum'].astype(np.int)
    df['travelers_sum'] = df['travelers_sum'].astype(np.int)
    df['star'] = df['star'].astype(np.int)
    #Fill in a null value, 0 if the column is of type Int; If it's str, then nan
    df.fillna(value = 0,inplace = True) 
    return df

# Filter the data of the top n in the comprehensive score
def data_top(urls,n):  # The first n Numbers
    df = get_all_data(urls)
    df = data_collation(df)
    # Constructor function to achieve field standardization, standard points
    cols = ['strategy_sum','star','comment_sum']
    for col in cols:
        df[col + '_b'] = round((df[col] - df[col].min())/(df[col].max() - df[col].min())*100,2)            
    #Percentage score + number of mentions in the strategy + star score + number of comments score, each item is 0-100
    df['overall_score'] = df['travelers_sum']+df['strategy_sum_b']+df['star_b']+df['comment_sum_b']
    top_n = df.sort_values(by = 'overall_score', ascending=False).iloc[:n]    
    del top_n['strategy_sum_b']
    del top_n['comment_sum_b']
    del top_n['star_b']
    return top_n

# Climb the scenic spot link to get scenic spot details
def get_onepage_info(name,u): #u:url
    r = requests.get(u)
    soup = BeautifulSoup(r.text, 'lxml')
    dic_jd = {}
    dic_jd['attraction_names'] = name
          
    dic_jd['visit_time'] = soup.find('div',class_ = "txtbox")
    # Determine the null value to prevent errors
    if dic_jd['visit_time'] is None: 
        dic_jd['visit_time'] = 'none'
    else:
        dic_jd['visit_time'] = dic_jd['visit_time'].find('div',class_ ='time').text 
        
    dic_jd['ticket_price'] = soup.find('div',class_ = "b_detail_section b_detail_ticket")
    if dic_jd['ticket_price'] is None:  #判断空值，防止出错
        dic_jd['ticket_price'] = '无'
    else:
        dic_jd['ticket_price'] = dic_jd['ticket_price'].find('div',class_ = "e_db_content_box e_db_content_dont_indent").text   
    
    
    return dic_jd

# Get details of all attractions by crawling the links
def get_detailed(df):
    detailed_data = []
    for index,row in df.iterrows():
        detailed_data.append(get_onepage_info(index,row['url']))
    detailed_df = pd.DataFrame(detailed_data)
    detailed_df.index = detailed_df['attraction_names']
    del detailed_df['attraction_names']
 # In order to make the data more concise, it is necessary to note that all data points to the same data set, and the data in the previous df is not available after deletion
    del df['url']
    del df['strategy_sum']
    del df['comment_sum']
    del df['star']
    del df['travelers_sum']
    res = pd.concat([df,detailed_df],axis= 1)      #Add site details
    return res

if __name__=="__main__":
    start_time = time.time()
    urls = get_urls('https://travel.qunar.com/p-cs65527-jianqiao-jingdian-1-',5) # Links and pages
    top30_data = data_top(urls,30)   # The data of top 30
    top30_data.to_excel('Cambridge.xlsx')      # export file
    end_time1 = time.time()
    print ("crawl the basic data，it takes:",end_time1 - start_time)    
    
    res = get_detailed(top30_data)      # Add detailed scenic spot information, compare the cost of time
    res.to_excel('Cambridge.xlsx')     # export file
    end_time2 = time.time()
    print ("crawl the basic data，it takes:",end_time2 - start_time)

crawl the basic data，it takes: 10.618989706039429
